In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE280643"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE280643"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE280643.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE280643.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE280643.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential KEAP1/NRF2-mediated signaling widens the therapeutic window of redox-targeting drugs in SCLC therapy"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: normal lung', 'tissue: small cell lung cancer', 'tissue: normal skin']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Since this is a SuperSeries composed of SubSeries, and the sample characteristics
# indicates tissue types including normal lung and small cell lung cancer,
# it's likely that gene expression data is included in one of the SubSeries
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we see only one key (0)
# The values are tissue types: normal lung, small cell lung cancer, normal skin
# We can use this to infer our trait (lung cancer) - samples with "small cell lung cancer" are cases (1)
# and samples with "normal lung" are controls (0)
trait_row = 0  # Using the available row for tissue type

# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert tissue type to binary trait value (1 for cancer, 0 for normal lung)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on tissue type
    if "small cell lung cancer" in value.lower():
        return 1  # Case: has lung cancer
    elif "normal lung" in value.lower():
        return 0  # Control: normal lung tissue
    else:
        return None  # Other tissue types like normal skin are not relevant

def convert_age(value):
    """Placeholder function as age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function as gender data is not available"""
    return None

# 3. Save Metadata
# The trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we should extract clinical features
if is_trait_available:
    # Assuming clinical_data is defined from a previous step
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,  # This should be defined in a previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM8602788': [0.0], 'GSM8602789': [0.0], 'GSM8602790': [0.0], 'GSM8602791': [0.0], 'GSM8602792': [0.0], 'GSM8602793': [0.0], 'GSM8602794': [1.0], 'GSM8602795': [1.0], 'GSM8602796': [1.0], 'GSM8602797': [1.0], 'GSM8602798': [1.0], 'GSM8602799': [1.0], 'GSM8602800': [nan], 'GSM8602801': [nan], 'GSM8602802': [nan], 'GSM8602803': [nan], 'GSM8602804': [nan], 'GSM8602805': [nan], 'GSM8602806': [1.0], 'GSM8602807': [1.0], 'GSM8602808': [1.0], 'GSM8602809': [1.0], 'GSM8602810': [1.0], 'GSM8602811': [1.0]}
Clinical data saved to ../../output/preprocess/Lung_Cancer/clinical_data/GSE280643.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Differential KEAP1/NRF2-mediated signaling widens the therapeutic window of redox-targeting drugs in SCLC therapy"
Line 1: !Series_geo_accession	"GSE280643"
Line 2: !Series_status	"Public on Nov 11 2024"
Line 3: !Series_submission_date	"Oct 30 2024"
Line 4: !Series_last_update_date	"Nov 12 2024"
Line 5: !Series_web_link	"https://doi.org/10.1101/2024.11.06.621846"
Line 6: !Series_summary	"This SuperSeries is composed of the SubSeries listed below."
Line 7: !Series_overall_design	"Refer to individual Series"
Line 8: !Series_type	"Expression profiling by high throughput sequencing"
Line 9: !Series_type	"Expression profiling by array"
Found table marker at line 63
First few lines after marker:
"ID_REF"	"GSM8602788"	"GSM8602789"	"GSM8602790"	"GSM8602791"	"GSM8602792"	"GSM8602793"	"GSM8602794"	"GSM8602795"	"GSM8602796"	"GSM8602797"	"GSM8602798"	"GSM8602799"	"GSM8602800"	"GSM8602801"	"GSM8602802"	"GSM8602803"	"GSM8602804"	"GSM8602805"	

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# IDs like '23064070', '23064071', etc. are numeric identifiers, likely probe IDs
# These are not standard human gene symbols (which would typically be like 'BRCA1', 'TP53', etc.)
# These appear to be Illumina or Affymetrix probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 679749 rows

Gene annotation preview (first few rows):
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 

### Step 6: Gene Identifier Mapping

In [7]:
# Examining the gene expression data and annotation data
# From previous steps:
# - Gene expression data has IDs like '23064070', '23064071', etc.
# - We need to map these probe IDs to gene symbols

# First, let's search directly in the SOFT file for the probe-to-gene mapping
print("Looking for probe-to-gene mapping in the SOFT file...")
print("Sample probe IDs from expression data:")
print(gene_data.index[:5].tolist())

# Let's extract platform information first
platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith("!Series_platform_id"):
            platform_id = line.split("=")[1].strip().strip('"')
            break
print(f"Dataset uses platform: {platform_id}")

# Now let's directly search for probe-to-gene mappings in the SOFT file
# We're looking for sections that define relations between probe IDs and gene symbols
with gzip.open(soft_file, 'rt') as f:
    content = f.read()
    
    # Search for ID definitions that match our expression data probe IDs
    probe_mappings = []
    
    # Try to find the platform table sections which typically contain gene mappings
    platform_sections = re.findall(r'^\^PLATFORM.+?(?=^\^|\Z)', content, re.MULTILINE | re.DOTALL)
    
    if platform_sections:
        print(f"Found {len(platform_sections)} platform sections")
        
        # Process each platform section
        for section in platform_sections:
            # Check for a table with gene info
            table_match = re.search(r'!platform_table_begin(.+?)!platform_table_end', section, re.DOTALL)
            if table_match:
                table_content = table_match.group(1)
                lines = table_content.strip().split('\n')
                
                # Get header line to identify columns
                header = lines[0].split('\t')
                
                # Find ID column and gene symbol column
                id_col = -1
                symbol_col = -1
                
                for i, col_name in enumerate(header):
                    if col_name == 'ID':
                        id_col = i
                    elif 'GENE' in col_name.upper() or 'SYMBOL' in col_name.upper():
                        symbol_col = i
                
                if id_col >= 0 and symbol_col >= 0:
                    print(f"Found ID column at {id_col} and symbol column at {symbol_col}")
                    # Process data rows
                    for line in lines[1:]:
                        cols = line.split('\t')
                        if len(cols) > max(id_col, symbol_col):
                            probe_id = cols[id_col]
                            gene_symbol = cols[symbol_col]
                            if gene_symbol and probe_id:
                                symbols = extract_human_gene_symbols(gene_symbol)
                                if symbols:
                                    probe_mappings.append((probe_id, symbols))
    
    # If we didn't find a proper table, try a more aggressive approach
    # Look for ID lines in the entire file
    if not probe_mappings:
        print("No structured mapping found, searching for probe ID definitions...")
        # Get a sample of probe IDs we need to match
        sample_ids = gene_data.index[:10].tolist()
        
        for probe_id in sample_ids:
            # Look for lines containing this probe ID
            probe_matches = re.findall(r'[^\w]' + re.escape(probe_id) + r'[^\w](.+?)$', content, re.MULTILINE)
            for match in probe_matches:
                symbols = extract_human_gene_symbols(match)
                if symbols:
                    probe_mappings.append((probe_id, symbols))
                    break

# If we found mappings, create the mapping dataframe
if probe_mappings:
    mapping_df = pd.DataFrame(probe_mappings, columns=['ID', 'Gene'])
    print(f"Found {len(mapping_df)} probe-to-gene mappings")
    print("Sample of extracted mappings:")
    print(preview_df(mapping_df))
else:
    print("Could not find direct probe-to-gene mappings.")
    
    # As a fallback, create a simple mapping using numeric probe IDs only
    # This will at least allow data processing to continue
    probe_ids = gene_data.index.tolist()
    print(f"Using {len(probe_ids)} probe IDs directly as gene identifiers")
    
    # Create a simple mapping - each probe ID represents one "gene"
    # Map the ID to itself (as a list with one element, as expected by apply_gene_mapping)
    mapping_df = pd.DataFrame({'ID': probe_ids, 'Gene': [[id] for id in probe_ids]})
    print("Using probe IDs as gene identifiers")
    print(preview_df(mapping_df))

# Apply the gene mapping to convert probe-level data to gene expression data
print(f"\nApplying gene mapping to {len(gene_data)} probe measurements...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Apply gene symbol normalization to handle gene synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Converted to gene expression data with {len(gene_data.index)} genes")
if len(gene_data.index) > 0:
    print("First 20 genes:")
    print(gene_data.index[:20])
else:
    print("WARNING: No genes were mapped successfully!")
    # In this case, use the original probe IDs directly
    gene_data = gene_data.rename_axis('Gene')
    print(f"Using original {len(gene_data)} probe measurements as gene data")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Looking for probe-to-gene mapping in the SOFT file...
Sample probe IDs from expression data:
['23064070', '23064071', '23064072', '23064073', '23064074']
Dataset uses platform: GPL23159


Found 3 platform sections
No structured mapping found, searching for probe ID definitions...


Could not find direct probe-to-gene mappings.
Using 27189 probe IDs directly as gene identifiers
Using probe IDs as gene identifiers
{'ID': ['23064070', '23064071', '23064072', '23064073', '23064074'], 'Gene': [['23064070'], ['23064071'], ['23064072'], ['23064073'], ['23064074']]}

Applying gene mapping to 27189 probe measurements...
Converted to gene expression data with 0 genes
Using original 0 probe measurements as gene data
Gene expression data saved to ../../output/preprocess/Lung_Cancer/gene_data/GSE280643.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 0 unique genes remain
Normalized gene expression data saved to ../../output/preprocess/Lung_Cancer/gene_data/GSE280643.csv

Extracting clinical data directly from the matrix file...

Processing clinical data...
Extracted clinical data with shape: (1, 24)
Clinical data saved to: ../../output/preprocess/Lung_Cancer/clinical_data/GSE280643.csv

Linking clinical and genetic data...
Found 24 common samples between clinical and genetic data
Linked data shape: (24, 1)

Handling missing values in linked data...
Data shape after handling missing values: (0, 1)

Evaluating trait and demographic feature bias...
Quartiles for 'Lung_Cancer':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Lung_Cancer' in this dataset is fine.

Abnormality detected in the cohort: GSE280643. Preprocessing failed.



Dataset not usable for Lung_Cancer association studies due to bias or quality issues.
